In [191]:
from elasticsearch_dsl import connections, Index, analyzer, DocType, Date, Text, Keyword, Search, Nested, InnerDoc
import json
import pprint
import os

In [2]:
# in case of problsems with "only allow delete api" then:
# curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_all/_settings -d '{"index.blocks.read_only_allow_delete": null}'

Defines a default connection that will be used globally

In [3]:
connections.create_connection(hosts=['localhost'], timeout=20) 

<Elasticsearch([{'host': 'localhost'}])>

Defines an index

In [4]:
judgments = Index(name = 'judgments')

In [130]:
# TODO: resolve what should be datatype for judges
class Judge(InnerDoc):
    function = Text()
    name = Text()
    specialRoles = Text()


class Judgments(DocType):
    content = Text()
    date = Date()
    title = Keyword()
    judges = Nested(Judge)
    
    class Meta:
        index = 'title'
    
    def add_judge(self, function, name, specialRoles):
        self.judges.append(
            Judge(function=function, name=name, specialRoles=specialRoles))
    

In [165]:
judgments.doc_type(Judgments)

__main__.Judgments

In [166]:
default_analyzer = analyzer('default_analyzer',
                           tokenizer='standard')

In [167]:
judgments.analyzer(default_analyzer)

In [168]:
judgments.delete(ignore=404)
judgments.create()

In [169]:
def read_file(file_path):
    with open (file_path) as file:
        json_content = json.load(file)
        for judgment in json_content:
            content = judgment['textContent']
            date = judgment['judgmentDate']
            title = judgment['courtCases'][0]['caseNumber']
            judges  = judgment['judges']
            # pprint.pprint(judges)
            maping = Judgments(content=content, date=date, title=title)
            for judge in judges:
                maping.add_judge(function=judge['function'], 
                             name=judge['name'], 
                             specialRoles=judge['specialRoles'])
                maping.save()
#             if not judges:
#                 maping.add_judge(function="", name="", specialRoles="")
#                 maping.save()

In [170]:
def read_all_judgments_from_2018():
    for filename in os.listdir("../data_filtered/"):
        read_file("../data_filtered/" + filename)

In [171]:
read_all_judgments_from_2018()

In [190]:
query = Search().query("match", content="szkoda")
response = query.execute(ignore_cache=True)
print (query.to_dict())
print (f"Liczba orzeczen w ktorych wystepuje slowo szkoda wynosi: {query.count()}")

{'query': {'match': {'content': 'szkoda'}}}
Liczba orzeczen w ktorych wystepuje slowo szkoda wynosi: 550


In [ ]:
query = Search().query("").fil